# pydlem.prep Package Example
This notebook provides a complete example of acquiring meterology data and combining it with reservoir data to produce a formatted input data file for running the pydlem model.

First import necessary packages/modules

In [1]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np


pydlem_dir = r'C:\Users\CNB968\OneDrive - MT\GitHub\pydlem'
os.chdir(pydlem_dir)

from prep.datafile import CreateInputFile
from prep.lakegeom import area_from_eac_curve
from prep.lakegeom import calc_lake_depth
from prep.lakegeom import calc_fetch_length
from prep.metdata import calculate_vpd
from prep.metdata import get_gridmet_at_points
from prep.datafile import check_format

Initializing pydlem.prep module.


## Example 1: Static Reservoir Inputs
For this example, we will load the geometry data and extract GridMET meterology for the lakes of interest. In this case, a shapefile is used but this could be extracted from a database or any other method supported by Geopandas.

In [2]:
exres_pth = Path(r'D:\ArcGIS_Projects\Yellowstone\Tongue River\Vector\HWWY_reservoirs_5070.shp')
exres = gpd.read_file(exres_pth)
# this file is in crs 5070, need it to be 4326 for getting GridMET
exres = exres.to_crs(4326)

The default behavior of the CreateInputFile class is to download a complete POR series of meteorology from GridMET. As of right now, if you want a shorter period, use the get_gridmet_at_points() function directly.

In [3]:
# This example provides a unique identifier column rather than using the default DataFrame index.
# We extract just 15 days of meterology data for this example. This function is slow for a long period of record over many sites.
exres_met = get_gridmet_at_points(exres, 'Permanent_', start='2022-06-01', end='2022-06-15')

C:\Users\CNB968\OneDrive - MT\GitHub\pydlem\prep\metdata.py:51: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))


Retrieving GridMET cells...


C:\Users\CNB968\OneDrive - MT\GitHub\pydlem\prep\metdata.py:72: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid


8 unique GridMET cells found for 11 input features.
Fetching GridMET data for unique cells...


Cells: 100%|██████████| 8/8 [01:58<00:00, 14.82s/it]


In [4]:
exres_met

<xarray.Dataset> Size: 10kB
Dimensions:   (time: 15, location: 11)
Coordinates:
    lat       (location) float64 88B 44.54 44.56 44.57 ... 44.5 44.53 44.55
    lon       (location) float64 88B -107.2 -107.2 -107.2 ... -107.2 -107.2
    elev      (location) float64 88B 2.638e+03 2.518e+03 ... 2.664e+03 2.618e+03
  * location  (location) <U9 396B '71772797' '71772129' ... '71772619'
  * time      (time) datetime64[ns] 120B 2022-06-01 2022-06-02 ... 2022-06-15
Data variables:
    precip    (time, location) float64 1kB 0.85 0.8 0.8 1.5 ... 0.0 0.0 0.0 0.0
    min_temp  (time, location) float64 1kB 267.7 269.2 269.1 ... 272.2 272.8
    max_temp  (time, location) float64 1kB 282.7 283.9 284.1 ... 285.3 286.1
    srad      (time, location) float64 1kB 273.5 271.9 271.9 ... 297.4 297.3
    wind_dir  (time, location) float64 1kB 294.0 294.0 294.0 ... 280.0 280.0
    wind_vel  (time, location) float64 1kB 2.35 2.3 2.4 2.0 ... 7.55 7.85 8.1
    vpd       (time, location) float64 1kB 0.38 0.43 0.44 ... 0.52 0.57 0.62
Attributes:
    featureType:  timeSeries

To demonstrate loading a netcdf into the CreateInputFile class, we will save this meteorology data as a netcdf and then create an input datafile from it.

In [15]:
exres_met.to_netcdf('prep_Example1_met.nc')

Lake Area and Depth are required inputs to create the datafile, as pydlem needs these to run the computations. So first, we will create these and then load inputs from our previos meteorlogy netcdf file. The easiest way to start with pydlem is to assume static values for reservoir variables 'Depth' and 'Surface Area.' We will create some static data for the 15 days of interest.

In [5]:
# surface area from NHD
lareas = np.array([exres[exres['Permanent_'] == x]['AreaSqKm'].iloc[0] for x in exres_met.location.values])
lareas_arr = np.tile(lareas, (len(exres_met.time + 1), 1))
# in this case we will make it an xarray DataArray
Lake_Area = xr.DataArray(lareas_arr, coords=exres_met.coords, attrs={'standard_name': 'Lake Surface Area', 'units': 'km^2'})
Lake_Area.name = 'LakeArea'

# Next, to get depth we need volume. You can estimate the volume or assume an average value for depth. In this case, the reservoirs have max capacity 
# data that are manually listed here and used to calcuate depth.

# These are in cubic kilometers
caps = np.array([0.000259, 0.0128, 0.00018, 0.0042, 0.000099, 0.00186, 0.00157, 0.00046, 0.00098, 0.0057, 0.00069])
caps_arr = np.tile(caps, (len(exres_met.time + 1), 1))
ldpth = (caps_arr / lareas_arr) * 1000 # convert to meters of depth
Lake_Depth = xr.DataArray(ldpth, coords=exres_met.coords, attrs={'standard_name': 'Lake Depth', 'units': 'm'})
Lake_Depth.name = 'LakeDepth'

Now we load our saved netcdf from file into xarray and create the input data file.

In [6]:
met_ff = xr.load_dataset('prep_Example1_met.nc')
nw_Idat = CreateInputFile(geoms=None, lake_area=Lake_Area, lake_depth=Lake_Depth, met_data=met_ff, met_source='from_file')

New variable added.
New variable added.


At this point, are we finished? We can check if all variables are there by using the check_format() function.

In [7]:
check_format(nw_Idat.data)

MISSING VARIABLES
mean_temp
ftch_len
All necessary coordinates exist and are labeled properly


C:\Users\CNB968\OneDrive - MT\GitHub\pydlem\prep\datafile.py:194: UserWarning: There are missing or mislabeled variables in the dataset. See the following:
  warnings.warn("There are missing or mislabeled variables in the dataset. See the following:", UserWarning)


We still are missing some required variables. mean_temp can be calculated from our GridMET derived min/max temperature and added to the dataset. 'ftch_len' is the FETCH for the individual lakes. This is an important term in the dlem model that accounts for wind blowing across the lakes surface. The lakegeom.py module within the pydlem.prep package has a method to calculate fetch length needed to run the model.

Next we will compute these 2 required variables and add them to the dataset using the class method add_variable().

In [8]:
# mean temp
mean_temp = ((nw_Idat.data.min_temp + nw_Idat.data.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
nw_Idat.add_variable(mean_temp, variable_name='mean_temp', var_attrs={'standard_name': 'Average Daily Temperature', 'units': 'Celcius'})

New variable added.


In [9]:
# calculating the fetch length
# fetch is very dynamic, it depends on lake area, wind direction, and surface area geometry which are always flucuating
# thus, the fetch length function calculates only one fetch length at a time, so input arrays need to be looped over their 2 dimensions
# (time, location) calculating fetch length for all values

# assign necessary variables and complete any conversions
rows = nw_Idat.data.wind_dir.shape[0]
cols = nw_Idat.data.wind_dir.shape[1]
wnd_d = nw_Idat.data.wind_dir.values
lk_A = nw_Idat.data.LakeArea.values * 1000.0**2  # convert to meters
# Loop through data arrays of wind direction for each timestamp(x) and location(y)
resvs = exres.to_crs(exres.estimate_utm_crs())
xs = []
for x in range(0, rows):
    ys = []
    for y in range(0, cols):
        gmtry = resvs.loc[resvs['Permanent_'] == nw_Idat.data.location[y].item()].geometry
        wnd = wnd_d[x, y]
        A = lk_A[x, y]
        ftchL = calc_fetch_length(gmtry, wnd, A)
        ys.append(ftchL)
    xs.append(ys)
# create xarray DataArray from the resulting ndarray
flen = np.array(xs)
flen = xr.DataArray(flen, coords=nw_Idat.data.coords, attrs={'standard_name': 'Wind Fetch Length', 'units': 'm'})
# Add to the dataset
nw_Idat.add_variable(flen, 'fetch')

ValueError: Variable name not compatible, select from: ['precip', 'min_temp', 'max_temp', 'mean_temp', 'solrad', 'min_rh', 'max_rh', 'mean_rh', 'dew_temp','wind_dir', 'wind_vel', 'LakeArea', 'LakeDepth', 'vpd', lrad, 'ftch_len']

Uh oh! Something went wrong. This shows how variable addition is regulated within the CreateInputFile class, we put a variable name of
'fetch' which is not one of the available options. This is important because pydlem uses the variable names in all its calculations, that is why
this is strictly enforced!

Let's try again with the proper variable name = 'ftch_len'

In [10]:
nw_Idat.add_variable(flen, 'ftch_len')

New variable added.


In [11]:
check_format(nw_Idat.data)

All necessary variables exist and are labeled properly.
All necessary coordinates exist and are labeled properly


We now have a successfully built input file for pydlem. We can save it as Example 1 for use in the model running example notebook.

In [12]:
nw_Idat.save_datafile('Example1_static_inputs.nc')